In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.nn.parameter import Parameter
# import Transforms
%matplotlib inline
import pylab as pl
from IPython import display
import numpy as np
import time

In [2]:
data = torch.load("../mIN_train.pth")

In [3]:
torch.cuda.set_device(0)
a = 512
newdata = torch.stack([torch.ByteTensor(data[0][i]).float().view(84,84,3).transpose(0,2).contiguous() 
                       for i in range(64)],0)
newdata = Variable(newdata).cuda()
criterion = nn.MSELoss()
targ = Variable(torch.randn(64,a,16,16)).cuda()

In [4]:
class testnet(nn.Module):
    def __init__(self):
        super(testnet, self).__init__()
        self.process = nn.Sequential(
            nn.MaxPool2d(5),
            nn.Conv2d(3,a,(1,1),padding=0),
            nn.ReLU(),
            nn.Conv2d(a,a,(3,3),padding=1),
            nn.ReLU(),
            nn.Conv2d(a,a,(3,3),padding=1),
            nn.ReLU(),
            nn.Conv2d(a,a,(3,3),padding=1),
            nn.ReLU(),
            nn.Conv2d(a,a,(3,3),padding=1),
            nn.ReLU(),
            nn.Conv2d(a,a,(3,3),padding=1),
            nn.ReLU(),
            nn.Conv2d(a,a,(3,3),padding=1),
            nn.ReLU()
        )
        
    def forward(self, inp):
        return self.process(inp)
    
net = testnet().cuda()
nweights = sum([i.numel() for i in list(net.parameters())])
print(nweights," parameters in baseline neural net.")

14160896  parameters in baseline neural net.


In [5]:
# TEST
start = time.time()
for i in range(1):
    net.zero_grad()
    out = net(newdata)
    loss = criterion(out,targ)
    loss.backward()
    torch.cuda.synchronize()
print(time.time()-start)
# 2367 Mb  64
# 4241 Mb  128
# 9169 Mb  256

# 779 Mb  256 16
# 3291 Mb  512 16

0.8953635692596436
